In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from matplotlib import pyplot
from numpy import array
from sklearn.model_selection import train_test_split

from sklearn.metrics import *
from keras import backend
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Activation, BatchNormalization
from keras import regularizers
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import LeakyReLU
import random
from numpy.random import seed

from funcionesAux import *

In [2]:
indice_entrenamiento = 83
indice_test = 108
cant_pruebas = 10
cant_meses = 24

path = "resultados_redes_neuronales_obligatorio\\"

columna_objetivo ='consumo_miles_m3(t)'

#ordenados de menor a mayor

lrs = [0.0007]
batchs = [5]
epochss = [300]

arquitecturas = [([200, 150, 100], [50, 25])]

experimentos = crearExperimentos(lrs, batchs, epochss, arquitecturas)

tf.random.set_seed(40)
random.seed(40)
np.random.seed(40)


In [3]:
#cant_meses = indice_test - indice_entrenamiento

In [4]:
series = pd.read_csv("gasoil_csv.csv",sep = ',',header = 0)
meses = pd.read_csv("meses.csv",sep = ',',header = 0)
diasVentasRC = pd.read_csv("dias_venta_teoricos.csv",sep =',',header = 0)

In [5]:
#ESCALAMIENTO DE MESES
scaler = MinMaxScaler(feature_range = (0, 1))
df_scaled = pd.DataFrame(scaler.fit_transform(meses), columns = meses.columns)
df_scaled

#ESCALAMIENTO DE DIAS DE VENTAS reales
scaler = MinMaxScaler(feature_range = (0, 1))
df_scaled2 = pd.DataFrame(scaler.fit_transform(diasVentasRC), columns = diasVentasRC.columns)
df_scaled2

#UNION DE DATAFRAME "series" y "meses escalado" y "dias de venta teoricos escalados"
seriesF=pd.concat([series, df_scaled], axis = 1)
seriesF=pd.concat([seriesF, df_scaled2], axis = 1)
seriesF


,Fecha,consumo_miles_m3,Meses,Dias de Venta
0,31/1/2010,68.874,0.000000,1.000000
1,28/2/2010,60.128,0.090909,0.000000
2,31/3/2010,84.494,0.181818,0.333333
3,30/4/2010,79.147,0.272727,0.666667
4,31/5/2010,72.691,0.363636,1.000000
...,...,...,...,...
127,31/8/2020,70.691,0.636364,1.000000
128,30/9/2020,73.769,0.727273,0.000000
129,31/10/2020,81.305,0.818182,1.000000
130,30/11/2020,85.335,0.909091,0.666667


In [6]:
n=24
series=series_to_supervised(seriesF,n,1)
series.head()

series=series.drop('Fecha(t-%d)' % 1, axis=1)
for i in range(2,n+1):
    aux='Fecha(t-%d)' % i
    series = series.drop(aux, axis=1)
series.head()

aux1 ='Dias de Venta(t)' 
series = series.drop(aux1, axis=1)

aux2 ='Meses(t)' 
series = series.drop(aux2, axis=1)

series = series.reset_index()
series

,index,consumo_miles_m3(t-24),Meses(t-24),Dias de Venta(t-24),consumo_miles_m3(t-23),Meses(t-23),Dias de Venta(t-23),consumo_miles_m3(t-22),Meses(t-22),Dias de Venta(t-22),...,Meses(t-3),Dias de Venta(t-3),consumo_miles_m3(t-2),Meses(t-2),Dias de Venta(t-2),consumo_miles_m3(t-1),Meses(t-1),Dias de Venta(t-1),Fecha(t),consumo_miles_m3(t)
0,24,68.874,0.000000,1.000000,60.128,0.090909,0.000000,84.494,0.181818,0.333333,...,0.818182,1.000000,90.184,0.909091,0.000000,82.524,1.000000,1.000000,31/1/2012,65.557
1,25,60.128,0.090909,0.000000,84.494,0.181818,0.333333,79.147,0.272727,0.666667,...,0.909091,0.000000,82.524,1.000000,1.000000,65.557,0.000000,0.666667,29/2/2012,68.499
2,26,84.494,0.181818,0.333333,79.147,0.272727,0.666667,72.691,0.363636,1.000000,...,1.000000,1.000000,65.557,0.000000,0.666667,68.499,0.090909,0.000000,31/3/2012,76.230
3,27,79.147,0.272727,0.666667,72.691,0.363636,1.000000,69.217,0.454545,0.000000,...,0.000000,0.666667,68.499,0.090909,0.000000,76.230,0.181818,1.000000,30/4/2012,79.642
4,28,72.691,0.363636,1.000000,69.217,0.454545,0.000000,66.817,0.545455,1.000000,...,0.090909,0.000000,76.230,0.181818,1.000000,79.642,0.272727,0.666667,31/5/2012,81.502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,127,72.907,0.636364,0.666667,68.894,0.727273,0.666667,85.930,0.818182,0.333333,...,0.363636,1.000000,66.408,0.454545,0.333333,72.149,0.545455,0.666667,31/8/2020,70.691
104,128,68.894,0.727273,0.666667,85.930,0.818182,0.333333,86.293,0.909091,0.666667,...,0.454545,0.333333,72.149,0.545455,0.666667,70.691,0.636364,1.000000,30/9/2020,73.769
105,129,85.930,0.818182,0.333333,86.293,0.909091,0.666667,71.868,1.000000,1.000000,...,0.545455,0.666667,70.691,0.636364,1.000000,73.769,0.727273,0.000000,31/10/2020,81.305
106,130,86.293,0.909091,0.666667,71.868,1.000000,1.000000,63.058,0.000000,0.333333,...,0.636364,1.000000,73.769,0.727273,0.000000,81.305,0.818182,1.000000,30/11/2020,85.335


In [7]:
train = series.loc[:indice_entrenamiento]
test_1 = series.loc[indice_entrenamiento+1:indice_test]

In [8]:
inputcols = ["Dias de Venta(t-24)","consumo_miles_m3(t-24)","Meses(t-24)",
"Dias de Venta(t-23)","consumo_miles_m3(t-23)","Meses(t-23)","Dias de Venta(t-22)",
"consumo_miles_m3(t-22)","Meses(t-22)","Dias de Venta(t-21)","consumo_miles_m3(t-21)","Meses(t-21)",
"Dias de Venta(t-20)","consumo_miles_m3(t-20)","Meses(t-20)","Dias de Venta(t-19)",
"consumo_miles_m3(t-19)","Meses(t-19)","Dias de Venta(t-18)","consumo_miles_m3(t-18)","Meses(t-18)",
"Dias de Venta(t-17)","consumo_miles_m3(t-17)","Meses(t-17)","Dias de Venta(t-16)",
"consumo_miles_m3(t-16)","Meses(t-16)","Dias de Venta(t-15)","consumo_miles_m3(t-15)",
"Meses(t-15)","Dias de Venta(t-14)","consumo_miles_m3(t-14)","Meses(t-14)","Dias de Venta(t-13)",
"consumo_miles_m3(t-13)","Meses(t-13)","Dias de Venta(t-12)","consumo_miles_m3(t-12)",
"Meses(t-12)","Dias de Venta(t-11)","consumo_miles_m3(t-11)","Meses(t-11)","Dias de Venta(t-10)",
"consumo_miles_m3(t-10)","Meses(t-10)","Dias de Venta(t-9)","consumo_miles_m3(t-9)","Meses(t-9)",
"Dias de Venta(t-8)","consumo_miles_m3(t-8)","Meses(t-8)","Dias de Venta(t-7)","consumo_miles_m3(t-7)",
"Meses(t-7)","Dias de Venta(t-6)","consumo_miles_m3(t-6)","Meses(t-6)", "Dias de Venta(t-5)",
"consumo_miles_m3(t-5)","Meses(t-5)", "Dias de Venta(t-4)","consumo_miles_m3(t-4)","Meses(t-4)", 
"Dias de Venta(t-3)","consumo_miles_m3(t-3)","Meses(t-3)","Dias de Venta(t-2)","consumo_miles_m3(t-2)",
"Meses(t-2)","Dias de Venta(t-1)","consumo_miles_m3(t-1)","Meses(t-1)"]

etiqueta =["consumo_miles_m3(t)"]

In [9]:
x_train = train[inputcols].values
y_train = train[etiqueta].values

x_train = np.array(x_train)
x_train = np.reshape(x_train, (x_train.shape[0], n, int(x_train.shape[1]/n)))

In [10]:
x_test = test_1[inputcols].values
y_test = test_1[etiqueta].values

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], n, int(x_test.shape[1]/n)))

In [11]:
for e in experimentos:
    batch = e["batch"]
    lr = e["learning_rate"]
    epochs = e["epochs"]
    lstms = e["arquitectura"][0]
    densas = e["arquitectura"][1]

    input_s = (x_train.shape[1], x_train.shape[2])
    listAuxGasoil = []
   
    predGasoil = []
    mgasoil = []    
    for i in range(cant_pruebas):
        
        scoresMarianaGasoil = []

        modeloGasoil = crearModelo(input_s,lstms,densas,1, lr, True, 0.2, 0.001)  
        
        
        modeloGasoil.fit(x_train, y_train,epochs = epochs, batch_size = batch) 
        
        scoresGasoil = modeloGasoil.evaluate(x_test,y_test)
        predictionsGasoil = modeloGasoil.predict(x_test)

        scoresMarianaGasoil.append(scoresGasoil[1])

        predGasoil.append(predictionsGasoil)
        
        emensuales = erroresMensuales(cant_meses,predictionsGasoil,test_1[columna_objetivo])
        mgasoil.append(emensuales)
        
        listAuxGasoil.append(np.mean(emensuales))

    generacionArchivo(e, path,listAuxGasoil,cant_meses,predGasoil,mgasoil)
    


Epoch 1/300
17/17 [==============================] - 2s 90ms/step - loss: 75.8314 - rmse: 75.7373
Epoch 2/300
17/17 [==============================] - 2s 91ms/step - loss: 65.5615 - rmse: 65.4734
Epoch 3/300
17/17 [==============================] - 2s 90ms/step - loss: 48.2023 - rmse: 48.1146
Epoch 4/300
17/17 [==============================] - 1s 86ms/step - loss: 15.8408 - rmse: 15.7534
Epoch 5/300
17/17 [==============================] - 2s 89ms/step - loss: 11.0140 - rmse: 10.9267
Epoch 6/300
17/17 [==============================] - 1s 82ms/step - loss: 13.0810 - rmse: 12.9938
Epoch 7/300
17/17 [==============================] - 2s 89ms/step - loss: 14.7089 - rmse: 14.6219
Epoch 8/300
17/17 [==============================] - 1s 88ms/step - loss: 10.6624 - rmse: 10.5757
Epoch 9/300
17/17 [==============================] - 2s 89ms/step - loss: 10.1913 - rmse: 10.1048
Epoch 10/300
17/17 [==============================] - 1s 85ms/step - loss: 8.5508 - rmse: 8.4645
Epoch 11/300
17/17 [=

17/17 [==============================] - 2s 90ms/step - loss: 5.0524 - rmse: 4.9780
Epoch 86/300
17/17 [==============================] - 1s 83ms/step - loss: 4.5941 - rmse: 4.5200
Epoch 87/300
17/17 [==============================] - 2s 90ms/step - loss: 4.7135 - rmse: 4.6395
Epoch 88/300
17/17 [==============================] - 1s 85ms/step - loss: 4.5354 - rmse: 4.4615
Epoch 89/300
17/17 [==============================] - 2s 90ms/step - loss: 4.1613 - rmse: 4.0878
Epoch 90/300
17/17 [==============================] - 2s 89ms/step - loss: 4.1169 - rmse: 4.0433
Epoch 91/300
17/17 [==============================] - 2s 89ms/step - loss: 4.3333 - rmse: 4.2599
Epoch 92/300
17/17 [==============================] - 1s 83ms/step - loss: 4.2535 - rmse: 4.1803
Epoch 93/300
17/17 [==============================] - 1s 85ms/step - loss: 4.9986 - rmse: 4.9256
Epoch 94/300
17/17 [==============================] - 2s 90ms/step - loss: 5.5191 - rmse: 5.4461
Epoch 95/300
17/17 [=======================

17/17 [==============================] - 2s 89ms/step - loss: 3.9298 - rmse: 3.8652
Epoch 169/300
17/17 [==============================] - 1s 85ms/step - loss: 3.7585 - rmse: 3.6940
Epoch 170/300
17/17 [==============================] - 1s 86ms/step - loss: 4.5336 - rmse: 4.4691
Epoch 171/300
17/17 [==============================] - 1s 86ms/step - loss: 4.2014 - rmse: 4.1367
Epoch 172/300
17/17 [==============================] - 2s 89ms/step - loss: 3.6207 - rmse: 3.5560
Epoch 173/300
17/17 [==============================] - 1s 85ms/step - loss: 4.2357 - rmse: 4.1711
Epoch 174/300
16/17 [===========================>..] - ETA: 0s - loss: 5.9119 - rmse: 5.8473

KeyboardInterrupt: 